<a href="https://colab.research.google.com/github/jabir-al-nahian/custom_bert_chatbot/blob/main/custom_bert_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd;
df=pd.read_csv("/content/FAQs.csv");
df.columns=["questions","answers"];

df

,questions,answers
0,When was Albert Einstein born?,Albert Einstein was born on 14 March 1879.
1,Where was he born?,"He was born in Ulm, Germany."
2,When did he die?,"He died 18 April 1955 in Princeton, New Jersey..."
3,Who were his parents?,His father was Hermann Einstein and his mother...
4,Did he have any sisters and brothers?,He had one sister named Maja.
5,Did he marry and have children?,He was married to Mileva Marić between 1903 an...
6,Where did he receive his education?,He received his main education at the followin...
7,When was Albert Einstein awarded the Nobel Pri...,"The Nobel Prize Awarding Institution, the Roya..."
8,Did Albert Einstein attend the Nobel Prize Awa...,The Nobel Prize was announced on 9 November 19...
9,For what did he receive the Nobel Prize?,Einstein was rewarded for his many contributio...


In [ ]:
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords

#from nltk.stem.lancaster import LancasterStemmer
#st = LancasterStemmer()

def clean_sentence(sentence, stopwords=False):

    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
    #sentence = re.sub(r'\s{2,}', ' ', sentence)

    if stopwords:
         sentence = remove_stopwords(sentence)

    #sent_stemmed='';
    #for word in sentence.split():
    #    sent_stemmed+=' '+st.stem(word)
    #sentence=sent_stemmed

    return sentence

def get_cleaned_sentences(df,stopwords=False):
    sents=df[["questions"]];
    cleaned_sentences=[]

    for index,row in df.iterrows():
        #print(index,row)
        cleaned=clean_sentence(row["questions"],stopwords);
        cleaned_sentences.append(cleaned);
    return cleaned_sentences;

cleaned_sentences=get_cleaned_sentences(df,stopwords=True)
print(cleaned_sentences);

print("\n")

cleaned_sentences_with_stopwords=get_cleaned_sentences(df,stopwords=False)
print(cleaned_sentences_with_stopwords);

['albert einstein born', 'born', 'die', 'parents', 'sisters brothers', 'marry children', 'receive education', 'albert einstein awarded nobel prize physics', 'albert einstein attend nobel prize award ceremony', 'receive nobel prize']


['when was albert einstein born', 'where was he born', 'when did he die', 'who were his parents', 'did he have any sisters and brothers', 'did he marry and have children', 'where did he receive his education', 'when was albert einstein awarded the nobel prize in physics', 'did albert einstein attend the nobel prize award ceremony', 'for what did he receive the nobel prize']


In [ ]:
import numpy

sentences=cleaned_sentences_with_stopwords
#sentences=cleaned_sentences

# Split it by white space
sentence_words = [[word for word in document.split() ]
         for document in sentences]

from gensim import corpora

dictionary = corpora.Dictionary(sentence_words)
for key, value in dictionary.items():
    print(key, ' : ', value)

import pprint
bow_corpus = [dictionary.doc2bow(text) for text in sentence_words]
for sent,embedding in zip(sentences,bow_corpus):
    print(sent)
    print(embedding)

#question_orig="do I need to learn algorithms to be a data scientist ?";
question_orig="When was Albert Einstein born ?"
question=clean_sentence(question_orig,stopwords=False);
question_embedding = dictionary.doc2bow(question.split())


print("\n\n",question,"\n",question_embedding)

0  :  albert
1  :  born
2  :  einstein
3  :  was
4  :  when
5  :  he
6  :  where
7  :  did
8  :  die
9  :  his
10  :  parents
11  :  were
12  :  who
13  :  and
14  :  any
15  :  brothers
16  :  have
17  :  sisters
18  :  children
19  :  marry
20  :  education
21  :  receive
22  :  awarded
23  :  in
24  :  nobel
25  :  physics
26  :  prize
27  :  the
28  :  attend
29  :  award
30  :  ceremony
31  :  for
32  :  what
when was albert einstein born
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]
where was he born
[(1, 1), (3, 1), (5, 1), (6, 1)]
when did he die
[(4, 1), (5, 1), (7, 1), (8, 1)]
who were his parents
[(9, 1), (10, 1), (11, 1), (12, 1)]
did he have any sisters and brothers
[(5, 1), (7, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1)]
did he marry and have children
[(5, 1), (7, 1), (13, 1), (16, 1), (18, 1), (19, 1)]
where did he receive his education
[(5, 1), (6, 1), (7, 1), (9, 1), (20, 1), (21, 1)]
when was albert einstein awarded the nobel prize in physics
[(0, 1), (2, 1), (3, 1), 

In [ ]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity;
def retrieveAndPrintFAQAnswer(question_embedding,sentence_embeddings,FAQdf,sentences):
    max_sim=-1;
    index_sim=-1;
    for index,faq_embedding in enumerate(sentence_embeddings):
        #sim=cosine_similarity(embedding.reshape(1, -1),question_embedding.reshape(1, -1))[0][0];
        sim=cosine_similarity(faq_embedding,question_embedding)[0][0];
        print(index, sim, sentences[index])
        if sim>max_sim:
            max_sim=sim;
            index_sim=index;

    print("\n")
    print("Question: ",question)
    print("\n");
    print("Retrieved: ",FAQdf.iloc[index_sim,0])
    print(FAQdf.iloc[index_sim,1])

retrieveAndPrintFAQAnswer(question_embedding,bow_corpus,df,sentences);

0 1.0 when was albert einstein born
1 0.7071067811865475 where was he born
2 0.24253562503633297 when did he die
3 0.11043152607484653 who were his parents
4 0.19611613513818404 did he have any sisters and brothers
5 0.19611613513818404 did he marry and have children
6 0.19611613513818404 where did he receive his education
7 1.0 when was albert einstein awarded the nobel prize in physics
8 1.0 did albert einstein attend the nobel prize award ceremony
9 0.19611613513818404 for what did he receive the nobel prize


Question:  when was albert einstein born 


Retrieved:  When was Albert Einstein born?
Albert Einstein was born on 14 March 1879.


In [ ]:
from gensim.models import Word2Vec
import gensim.downloader as api


glove_model=None;
try:
    glove_model = gensim.models.KeyedVectors.load("./glovemodel.mod")
    print("Loaded glove model")
except:
    glove_model = api.load('glove-twitter-25')
    glove_model.save("./glovemodel.mod")
    print("Saved glove model")

v2w_model=None;
try:
    v2w_model = gensim.models.KeyedVectors.load("./w2vecmodel.mod")
    print("Loaded w2v model")
except:
    v2w_model = api.load('word2vec-google-news-300')
    v2w_model.save("./w2vecmodel.mod")
    print("Saved glove model")

w2vec_embedding_size=len(v2w_model['computer']);
glove_embedding_size=len(glove_model['computer']);

[==================================================] 100.0% 104.8/104.8MB downloaded
Saved glove model
[==================================================] 100.0% 1662.8/1662.8MB downloaded
Saved glove model


In [ ]:
def getWordVec(word,model):
        samp=model['computer'];
        vec=[0]*len(samp);
        try:
                vec=model[word];
        except:
                vec=[0]*len(samp);
        return (vec)


def getPhraseEmbedding(phrase,embeddingmodel):

        samp=getWordVec('computer', embeddingmodel);
        vec=numpy.array([0]*len(samp));
        den=0;
        for word in phrase.split():
            #print(word)
            den=den+1;
            vec=vec+numpy.array(getWordVec(word,embeddingmodel));
        #vec=vec/den;
        #return (vec.tolist());
        return vec.reshape(1, -1)

In [ ]:
sent_embeddings=[];
for sent in cleaned_sentences:
    sent_embeddings.append(getPhraseEmbedding(sent,v2w_model));

question_embedding=getPhraseEmbedding(question,v2w_model);

retrieveAndPrintFAQAnswer(question_embedding,sent_embeddings,df, cleaned_sentences);

0 0.8638502128845564 albert einstein born
1 0.6694598371680672 born
2 0.28243142756108536 die
3 0.25286840894071994 parents
4 0.30725008020188627 sisters brothers
5 0.38537646117043917 marry children
6 0.11739840966719121 receive education
7 0.43419905683258475 albert einstein awarded nobel prize physics
8 0.40819206508311107 albert einstein attend nobel prize award ceremony
9 0.22327438130221822 receive nobel prize


Question:  when was albert einstein born 


Retrieved:  When was Albert Einstein born?
Albert Einstein was born on 14 March 1879.


In [ ]:
sent_embeddings=[];
for sent in cleaned_sentences:
    sent_embeddings.append(getPhraseEmbedding(sent,glove_model));

question_embedding=getPhraseEmbedding(question,glove_model);

retrieveAndPrintFAQAnswer(question_embedding,sent_embeddings,df, cleaned_sentences);

0 0.9118603670721388 albert einstein born
1 0.9217135254907831 born
2 0.5944353315586562 die
3 0.6983396513630136 parents
4 0.791750140489032 sisters brothers
5 0.8680346743499505 marry children
6 0.6988570013955634 receive education
7 0.7518386980263269 albert einstein awarded nobel prize physics
8 0.7573800032533627 albert einstein attend nobel prize award ceremony
9 0.60394214852902 receive nobel prize


Question:  when was albert einstein born 


Retrieved:  Where was he born?
He was born in Ulm, Germany. 


In [ ]:
!pip install bert-serving-server
!pip install bert-serving-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 61 kB 272 kB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7410 sha256=8d973482bab422ef1e1ce51a26c92ac607c5904f7f58f7c61e8fa14512e5c6d0
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from bert_serving.client import BertClient
bc = BertClient()
res=bc.encode(['ML', 'AI'])

In [ ]:
question=clean_sentence(question_orig,stopwords=False);
cleaned_sentences=get_cleaned_sentences(df,stopwords=False)

sent_bertphrase_embeddings=[];
for sent in cleaned_sentences:
    sent_bertphrase_embeddings.append(bc.encode([sent]));

question_embedding=bc.encode([question]);

retrieveAndPrintFAQAnswer(question_embedding,sent_bertphrase_embeddings,df, cleaned_sentences);

In [ ]:
#With BOW, word2Vec and BERT with stopwords removed
question_new="What is the date of his death"

question=clean_sentence(question_new,stopwords=True);
cleaned_sentences=get_cleaned_sentences(df,stopwords=True)

#Try  BOW
#question_embedding = dictionary.doc2bow(question.lower().split())
#print("\n\n With BOW\n\n")
#retrieveAndPrintFAQAnswer(question_embedding,bow_corpus,df);

sent_embeddings=[];
for sent in cleaned_sentences:
    sent_embeddings.append(getPhraseEmbedding(sent,v2w_model));

question_embedding=getPhraseEmbedding(question,v2w_model);

print("\n With Word2Vec\n\n")
retrieveAndPrintFAQAnswer(question_embedding,sent_embeddings,df, cleaned_sentences);

question=clean_sentence(question_new,stopwords=False);
cleaned_sentences=get_cleaned_sentences(df,stopwords=False)

sent_bertphrase_embeddings=[];
for sent in cleaned_sentences:
    sent_bertphrase_embeddings.append(bc.encode([sent]));

question_embedding=bc.encode([question]);

print("\n\n With BERT\n\n")
retrieveAndPrintFAQAnswer(question_embedding,sent_bertphrase_embeddings,df, cleaned_sentences);